In [ ]:
from model_loader import ModelLoader
from text_utils import *
import pickle
from scipy import spatial

In [ ]:
# experiment_id, experiment_name
# 01daapwr6w051q9wwqy99jsgfy - Generic
# 01daaqy88qzb19jqz5prjfr76y - Engineering
# 01daaqyn9gbebc92aywnxedp0c - HR
# 01daatanxnrqa35e6004hb7mbn - Marketing
# 01daatbc3ak1qwc5nyc5ahv2xz - Product
# 01dadp74wfv607knpcb6vvxgtg - AI
# 01daayheky5f4e02qvrjptftxv - Ether Engineering

In [ ]:
config_path = '/Users/venkat/Documents/mlflow/mlflow_bert/bert_mlflow_pyfunc/artifacts/bert_config.json'
mind_path = '/Users/venkat/Documents/mlflow/mlflow_bert/bert_mlflow_pyfunc/artifacts/mind.pkl'
model_path = '/Users/venkat/Documents/mlflow/mlflow_bert_deploy/mind-01dadp74wfv607knpcb6vvxgtg/artifacts/model.bin'

model_loader = ModelLoader(config_path, mind_path, model_path)
model = model_loader.loadModel()

In [ ]:
#load test files
test_sents = pickle.load(open('/Users/venkat/Documents/Dev/sentence_relatedness/test_data/ai_minds_sample.pkl','rb'))
test_sents = list(test_sents['sentence'])

In [ ]:
sent_feat_list = []
for sent in test_sents:
    sent_feat_list.append(getBERTFeatures(model, sent, attn_head_idx=-1))

In [ ]:
kp_dist_list = []
cs_dist_list = []
text1_list = []
text2_list = []

for i in range(len(test_sents)):
    sent1 = test_sents[i]
    for j in range(i+1,len(test_sents)):
        sent2 = test_sents[j]
        text1_list.append(sent1)
        text2_list.append(sent2)
        tup1 = sent_feat_list[i]
        tup2 = sent_feat_list[j]
        #pos_dist_list.append(getPOSBasedSimilarity_loop(tup1, tup2, sent1, sent2))
        kp_dist_list.append(getKPBasedSimilarity(sent1, sent2, model, tup1, tup2))
        cs_dist_list.append(1-spatial.distance.cosine(tup1[1], tup2[1]))

In [ ]:
score_df = pd.DataFrame({'Sent1': text1_list,
                        'Sent2': text2_list,
                        'KP_Similarity': kp_dist_list,
                        'Cos_Similarity': cs_dist_list})

score_df['dist_diff'] = score_df['Cos_Similarity']-score_df['KP_Similarity']
score_df = score_df.sort_values(by='dist_diff',ascending=False)

In [ ]:
#score_df[score_df['dist_diff']<0.01]
score_df

In [ ]:
key = 21
text1 = score_df['Sent1'][key]
text2 = score_df['Sent2'][key]
layer_idx = 11
kp_sim = getKPBasedSimilarity(text1,text2,model,layer_idx)
cos_sim = getCosineSimilarity(text1, text2,model,layer_idx)